<img src="Layd_banner_02.png" 
     align="left"
     style="width:1000px;height:90px"/>

## Fetch Option chain data from NSE web site. Part-1
https://www.youtube.com/watch?v=Tke9SzbAqHc&t=1097s

### Import necessry library

In [1]:
import requests
import json
import pandas as pd

#### Set url tp scrap option chain data 
As of now here symbol is set as 'NIFTY' , this can be change for other symbpls as well e.g FINNIFTY or BANKNIFTY etc.
header setting also require else response will hang

In [2]:
url = 'https://www.nseindia.com/api/option-chain-indices?symbol=NIFTY'
headers={'user-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.127 Safari/537.36'}

#### Using requests library, get the data. 
Parameters are url and header. timeout is set to 10 seconds in case due to network issue there is delay in response 

In [3]:
response = requests.get(url, headers=headers, timeout=10)

response_text=response.text
json_object = json.loads(response_text)

In [4]:
json_object

{'records': {'expiryDates': ['25-Jan-2023',
   '02-Feb-2023',
   '09-Feb-2023',
   '16-Feb-2023',
   '23-Feb-2023',
   '02-Mar-2023',
   '29-Mar-2023',
   '29-Jun-2023',
   '28-Sep-2023',
   '28-Dec-2023',
   '27-Jun-2024',
   '26-Dec-2024',
   '26-Jun-2025',
   '24-Dec-2025',
   '25-Jun-2026',
   '31-Dec-2026',
   '24-Jun-2027',
   '30-Dec-2027'],
  'data': [{'strikePrice': 8500,
    'expiryDate': '29-Jun-2023',
    'CE': {'strikePrice': 8500,
     'expiryDate': '29-Jun-2023',
     'underlying': 'NIFTY',
     'identifier': 'OPTIDXNIFTY29-06-2023CE8500.00',
     'openInterest': 319.5,
     'changeinOpenInterest': 0,
     'pchangeinOpenInterest': 0,
     'totalTradedVolume': 0,
     'impliedVolatility': 0,
     'lastPrice': 1775,
     'change': 0,
     'pChange': 0,
     'totalBuyQuantity': 0,
     'totalSellQuantity': 150,
     'bidQty': 0,
     'bidprice': 0,
     'askQty': 150,
     'askPrice': 10642.25,
     'underlyingValue': 18027.65}},
   {'strikePrice': 9500,
    'expiryDate': '

## Fetch Option chain data from NSE web site. Part-2

#### Set the expiry date

In [5]:
expiry_date = '31-Dec-2026'

In [6]:
def set_decimal(x):
    return ('%.2f' % x).rstrip('0').rstrip('.')

In [7]:
set_decimal(10.234567)

'10.23'

In [8]:
data = json_object['records']['data']

In [9]:
len(data)

720

In [10]:
e_date=json_object['records']['expiryDates']

In [11]:
e_date

['25-Jan-2023',
 '02-Feb-2023',
 '09-Feb-2023',
 '16-Feb-2023',
 '23-Feb-2023',
 '02-Mar-2023',
 '29-Mar-2023',
 '29-Jun-2023',
 '28-Sep-2023',
 '28-Dec-2023',
 '27-Jun-2024',
 '26-Dec-2024',
 '26-Jun-2025',
 '24-Dec-2025',
 '25-Jun-2026',
 '31-Dec-2026',
 '24-Jun-2027',
 '30-Dec-2027']

In [12]:
oc_data = {}
for ed in e_date:
    oc_data[ed]={"CE":[], "PE":[]}
    for di in range(len(data)):
        
        if data[di]['expiryDate'] == ed:
            if 'CE' in data[di].keys() and data[di]['CE']['expiryDate'] == ed:                
                oc_data[ed]["CE"].append(data[di]['CE'])
            else:
                oc_data[ed]["CE"].append('-')

            if 'PE' in data[di].keys() and data[di]['PE']['expiryDate'] == ed:
                oc_data[ed]["PE"].append(data[di]['PE'])
                
            else:
                oc_data[ed]["PE"].append('-')

In [13]:
oc_data

{'25-Jan-2023': {'CE': [{'strikePrice': 14750,
    'expiryDate': '25-Jan-2023',
    'underlying': 'NIFTY',
    'identifier': 'OPTIDXNIFTY25-01-2023CE14750.00',
    'openInterest': 124,
    'changeinOpenInterest': 1,
    'pchangeinOpenInterest': 0.8130081300813008,
    'totalTradedVolume': 6,
    'impliedVolatility': 84.32,
    'lastPrice': 3309.85,
    'change': -53,
    'pChange': -1.5760441292356189,
    'totalBuyQuantity': 10000,
    'totalSellQuantity': 10000,
    'bidQty': 50,
    'bidprice': 3276.6,
    'askQty': 50,
    'askPrice': 3312.15,
    'underlyingValue': 18027.65},
   {'strikePrice': 14800,
    'expiryDate': '25-Jan-2023',
    'underlying': 'NIFTY',
    'identifier': 'OPTIDXNIFTY25-01-2023CE14800.00',
    'openInterest': 1,
    'changeinOpenInterest': 0,
    'pchangeinOpenInterest': 0,
    'totalTradedVolume': 0,
    'impliedVolatility': 0,
    'lastPrice': 3235.85,
    'change': 0,
    'pChange': 0,
    'totalBuyQuantity': 9900,
    'totalSellQuantity': 9900,
    'bidQ

In [14]:
keys_to_exclude = [
                    'pchangeinOpenInterest', 
                    'totalBuyQuantity', 
                    'totalSellQuantity', 
                    'underlyingValue',
                    'expiryDate',
                    'underlying',
                    'identifier',
                    'pChange'
]

In [15]:
oc_data_dt=oc_data[expiry_date]

CE=list(oc_data_dt['CE'])
PE=list(oc_data_dt['PE'])


In [16]:
for i in range(len(CE)):
    if CE[i] != '-':
        for key in keys_to_exclude:
            del CE[i][key]

    if PE[i] != '-':
        for key in keys_to_exclude:
            del PE[i][key]


In [17]:
CE[0]

'-'

In [18]:
PE[0]

{'strikePrice': 11000,
 'openInterest': 4,
 'changeinOpenInterest': 0,
 'totalTradedVolume': 0,
 'impliedVolatility': 0,
 'lastPrice': 431,
 'change': 0,
 'bidQty': 750,
 'bidprice': 1.75,
 'askQty': 0,
 'askPrice': 0}

In [19]:
l_OC = []

for i in range(len(CE)):
    l_CE=[]
    l_PE=[]

    if CE[i] != '-':
        sp = CE[i]['strikePrice']
        l_CE =  [  
                    CE[i]['openInterest'], 
                    CE[i]['changeinOpenInterest'] , 
                    CE[i]['totalTradedVolume'] , 
                    CE[i]['impliedVolatility'] , 
                    CE[i]['lastPrice'] , 
                    set_decimal(CE[i]['change']), 
                    CE[i]['bidQty']   , 
                    CE[i]['bidprice'] , 
                    CE[i]['askPrice'] ,
                    CE[i]['askQty'] , 
                    CE[i]['strikePrice']
                ]
    else:
        sp = PE[i]['strikePrice']
        l_CE= list(['-','-','-','-','-','-','-','-','-','-',sp])

    if PE[i] != '-':
        l_PE =  [
                    PE[i]['bidQty']   ,
                    PE[i]['bidprice'] ,
                    PE[i]['askPrice'] ,
                    PE[i]['askQty'] , 
                    set_decimal(PE[i]['change']),
                    PE[i]['lastPrice'] ,
                    PE[i]['impliedVolatility'] ,
                    PE[i]['totalTradedVolume'] ,
                    PE[i]['changeinOpenInterest'] ,
                    PE[i]['openInterest']
        ] 

    else:
        l_PE = list(['-','-','-','-','-','-','-','-','-','-'])
    
    
    l_OC_t = l_CE + l_PE
    l_OC_t[:] = [x if x != 0 else '-' for x in l_OC_t]

    l_OC.append(l_OC_t)


In [20]:
l_OC

[['-',
  '-',
  '-',
  '-',
  '-',
  '-',
  '-',
  '-',
  '-',
  '-',
  11000,
  750,
  1.75,
  '-',
  '-',
  '0',
  431,
  '-',
  '-',
  '-',
  4],
 ['-',
  '-',
  '-',
  '-',
  '-',
  '-',
  '-',
  '-',
  '-',
  '-',
  12000,
  750,
  5.3,
  '-',
  '-',
  '5',
  370,
  '-',
  '-',
  '-',
  9],
 ['-',
  '-',
  '-',
  '-',
  '-',
  '-',
  '-',
  '-',
  '-',
  '-',
  13000,
  750,
  3.75,
  '-',
  '-',
  '-5',
  535,
  '-',
  '-',
  '-',
  6],
 ['-',
  '-',
  '-',
  '-',
  '-',
  '0',
  '-',
  '-',
  9150,
  50,
  14000,
  50,
  400.05,
  '-',
  '-',
  '0',
  400.05,
  '-',
  '-',
  '-',
  16],
 ['-',
  '-',
  '-',
  '-',
  '-',
  '0',
  '-',
  '-',
  8615,
  50,
  15000,
  750,
  300.05,
  '-',
  '-',
  '0',
  800,
  '-',
  '-',
  '-',
  33],
 [1,
  '-',
  '-',
  '-',
  5450.25,
  '0',
  '-',
  '-',
  8110,
  50,
  16000,
  750,
  577.05,
  '-',
  '-',
  '0',
  850,
  '-',
  '-',
  '-',
  47],
 ['-',
  '-',
  '-',
  '-',
  '-',
  '0',
  '-',
  '-',
  7630,
  50,
  17000,
  750,
  500.0

In [21]:
OC_col = ['c_OI', 'c_CHNG_IN_OI', 'c_VOLUME', 'c_IV', 'c_LTP', 'c_CHNG', 'c_BID_QTY', 'c_BID', 'c_ASK', 'c_ASK_QTY',  'STRIKE', 'p_BID_QTY', 'p_BID', 'p_ASK', 'p_ASK_QTY',  'p_CHNG', 'p_LTP', 'p_IV', 'p_VOLUME', 'p_CHNG_IN_OI', 'p_OI'] 

In [22]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
df = pd.DataFrame(l_OC)
df.columns = OC_col

In [23]:
%%HTML
<style type="text/css">
    table.dataframe td, table.dataframe th {
        border-style: solid;
    }
</style>

In [24]:
df

,c_OI,c_CHNG_IN_OI,c_VOLUME,c_IV,c_LTP,c_CHNG,c_BID_QTY,c_BID,c_ASK,c_ASK_QTY,STRIKE,p_BID_QTY,p_BID,p_ASK,p_ASK_QTY,p_CHNG,p_LTP,p_IV,p_VOLUME,p_CHNG_IN_OI,p_OI
0,-,-,-,-,-,-,-,-,-,-,11000,750,1.75,-,-,0,431,-,-,-,4
1,-,-,-,-,-,-,-,-,-,-,12000,750,5.30,-,-,5,370,-,-,-,9
2,-,-,-,-,-,-,-,-,-,-,13000,750,3.75,-,-,-5,535,-,-,-,6
3,-,-,-,-,-,0,-,-,9150,50,14000,50,400.05,-,-,0,400.05,-,-,-,16
4,-,-,-,-,-,0,-,-,8615,50,15000,750,300.05,-,-,0,800,-,-,-,33
5,1,-,-,-,5450.25,0,-,-,8110,50,16000,750,577.05,-,-,0,850,-,-,-,47
6,-,-,-,-,-,0,-,-,7630,50,17000,750,500.05,-,-,0,1075,-,-,-,50
7,-,-,-,-,-,0,50,4200,7180,50,18000,50,1503.00,1639.9,50,0,1644.85,-,-,-,51
8,1,-,-,-,3830,0,50,2710,6740,50,19000,750,31.05,-,-,0,-,-,-,-,-
9,-,-,-,-,-,0,50,2173,6335,50,20000,50,1800.05,2899.9,50,559.95,2000,31.61,1,-,9
